In [1]:
import os

# Define the target directory (change yours)
target_directory = r"C:\Users\pablosal\Desktop\gbbai-semantic-kernel-nltosql"

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-semantic-kernel-nltosql


In [2]:
from dotenv import load_dotenv
import os
import asyncio
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from plugins.QueryDb import queryDb as plugin

kernel = sk.Kernel()

# Get AOAI settings from .env
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

In [3]:
# Set the deployment name to the value of your chat model
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
azure_text_service = AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key)
kernel.add_text_completion_service("azure_text_completion", azure_text_service)

# Immport NLP to SQL Plugin
plugins_directory = "plugins"
kernel.import_semantic_plugin_from_directory(plugins_directory, "nlpToSqlPlugin")
kernel.import_plugin(plugin.QueryDbPlugin(os.getenv("CONNECTION_STRING")), plugin_name="QueryDbPlugin")

KernelPlugin(name='QueryDbPlugin', description=None, functions={'query_db': KernelFunction(plugin_name='QueryDbPlugin', description='Query a database using a SQL query', name='query_db', is_semantic=False, stream_function=<bound method QueryDbPlugin.query_db of <plugins.QueryDb.queryDb.QueryDbPlugin object at 0x0000020E2ABB6FD0>>, parameters=[ParameterView(name='input', description='SQL Query to be executed', default_value='', type_='string', required=False)], delegate_type=<DelegateTypes.InKernelContextOutString: 5>, function=<bound method QueryDbPlugin.query_db of <plugins.QueryDb.queryDb.QueryDbPlugin object at 0x0000020E2ABB6FD0>>, plugins=KernelPluginCollection(plugins={'nlpToSqlPlugin': KernelPlugin(name='nlpToSqlPlugin', description=None, functions={'ConvertNLPToSQL': KernelFunction(plugin_name='nlpToSqlPlugin', description='Write SQL queries given a Natural Language description', name='ConvertNLPToSQL', is_semantic=True, stream_function=<function KernelFunction.from_semantic_co

In [4]:
async def create_plan(planner, input):
    return await planner.create_plan(goal=input)

async def invoke_plan(sequential_plan):
    return await sequential_plan.invoke()

In [5]:
import semantic_kernel as sk
from semantic_kernel.planning.sequential_planner import SequentialPlanner

In [ ]:
from semantic_kernel.planning.

In [6]:
planner = SequentialPlanner(kernel)


In [24]:
nlp_input= '''Can you list the UserId, ProfileName, Score, Summary, and Recommendation for users whose
 HelpfulnessNumerator divided by HelpfulnessDenominator is greater than 15?'''

In [25]:
# Create a plan with the NLP input
ask = f"Create a SQL query according to the following request: {nlp_input} and query the database to get the result."

plan = await create_plan(planner, ask)
 
# Invoke the plan and get the result
result = await invoke_plan(plan)

KernelContext(memory=NullMemory(), variables=ContextVariables(variables={'input': "Response: It appears there was an error with the database query. The system could not find the columns 'value1', 'value2', and 'value3'. Please check the column names and try again.", 'sql_query': 'The provided schema does not contain the necessary fields such as UserId, ProfileName, Score, Summary, Recommendation, HelpfulnessNumerator, and HelpfulnessDenominator to write the SQL query for the given objective. \n\nPlease provide the correct schema to proceed.', 'compatible_sql_query': 'INSERT INTO table_name (column1, column2, column3) VALUES (value1, value2, value3);', 'query_result': 'Error: (\'42S22\', "[42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name \'value1\'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name \'value2\'. (207); [42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name \'